In [1]:
import os
if os.path.split(os.getcwd())[-1]=='notebooks':
    os.chdir("../")

'Your base path is at: '+os.path.split(os.getcwd())[-1]

'Your base path is at: Lecture_Covid_19_data_analysis'

## 1 Update all data

In [2]:
# %load src/data/get_data.py

import subprocess
import os

import pandas as pd
import numpy as np

from datetime import datetime

import requests
import json

def set_data():
    data_confirmed = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
    data_confirmed = data_confirmed.drop(["Province/State", "Lat", "Long"], axis =1)
    data_confirmed = data_confirmed.groupby(["Country/Region"]).sum()

    recovered_data=pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv")
    recovered_data=recovered_data.drop(["Province/State","Lat", "Long"],axis=1)
    recovered_data = recovered_data.groupby(["Country/Region"]).sum()

    deaths_data=pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv")
    deaths_data=deaths_data.drop(["Province/State","Lat", "Long"],axis=1)
    deaths_data = deaths_data.groupby(["Country/Region"]).sum()

    return data_confirmed, recovered_data, deaths_data

confirmed_data, recovered_data, deaths_data = set_data()
str_dates = confirmed_data.columns
start_date = str_dates[0].split('/')
end_date = str_dates[-1].split('/')
if len(end_date[0]) == 1: 
    dates = np.arange(np.datetime64(f'20{start_date[2]}-0{start_date[0]}-{start_date[1]}'), np.datetime64(f'20{end_date[2]}-0{end_date[0]}-{int(end_date[1])+1}'))
else:
    dates = np.arange(np.datetime64(f'20{start_date[2]}-0{start_date[0]}-{start_date[1]}'), np.datetime64(f'20{end_date[2]}-{end_date[0]}-{int(end_date[1])+1}'))




## 2. Process pipeline 

In [3]:
# %load src/data/process_JH_data.py
import pandas as pd
import numpy as np

from datetime import datetime


def store_relational_JH_data():
    ''' Transformes the COVID data in a relational data set

    '''

    data_path='https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
    pd_raw=pd.read_csv(data_path)

    pd_data_base=pd_raw.rename(columns={'Country/Region':'country',
                      'Province/State':'state'})

    pd_data_base['state']=pd_data_base['state'].fillna('no')

    pd_data_base=pd_data_base.drop(['Lat','Long'],axis=1)


    pd_relational_model=pd_data_base.set_index(['state','country']) \
                                .T                              \
                                .stack(level=[0,1])             \
                                .reset_index()                  \
                                .rename(columns={'level_0':'date',
                                                   0:'confirmed'},
                                                  )

    pd_relational_model['date']=pd_relational_model.date.astype('datetime64[ns]')

    pd_relational_model.to_csv('data/processed/COVID_relational_confirmed.csv',sep=';',index=False)
    print(' Number of rows stored: '+str(pd_relational_model.shape[0]))
    print(' Latest date is: '+str(max(pd_relational_model.date)))
if __name__ == '__main__':

    store_relational_JH_data()


 Number of rows stored: 63042
 Latest date is: 2020-09-14 00:00:00


## 3  Filter and Doubling Rate Calculation

In [4]:
# %load src/features/build_features.py

import numpy as np
from sklearn import linear_model
reg = linear_model.LinearRegression(fit_intercept=True)
import pandas as pd

from scipy import signal


def get_doubling_time_via_regression(in_array):
    ''' Use a linear regression to approximate the doubling rate

        Parameters:
        ----------
        in_array : pandas.series

        Returns:
        ----------
        Doubling rate: double
    '''

    y = np.array(in_array)
    X = np.arange(-1,2).reshape(-1, 1)

    assert len(in_array)==3
    reg.fit(X,y)
    intercept=reg.intercept_
    slope=reg.coef_

    return intercept/slope


def savgol_filter(df_input,column='confirmed',window=5):
    ''' Savgol Filter which can be used in groupby apply function (data structure kept)

        parameters:
        ----------
        df_input : pandas.series
        column : str
        window : int
            used data points to calculate the filter result

        Returns:
        ----------
        df_result: pd.DataFrame
            the index of the df_input has to be preserved in result
    '''

    degree=1
    df_result=df_input

    filter_in=df_input[column].fillna(0) # attention with the neutral element here

    result=signal.savgol_filter(np.array(filter_in),
                           window, # window size used for filtering
                           1)
    df_result[str(column+'_filtered')]=result
    return df_result

def rolling_reg(df_input,col='confirmed'):
    ''' Rolling Regression to approximate the doubling time'

        Parameters:
        ----------
        df_input: pd.DataFrame
        col: str
            defines the used column
        Returns:
        ----------
        result: pd.DataFrame
    '''
    days_back=3
    result=df_input[col].rolling(
                window=days_back,
                min_periods=days_back).apply(get_doubling_time_via_regression,raw=False)



    return result




def calc_filtered_data(df_input,filter_on='confirmed'):
    '''  Calculate savgol filter and return merged data frame

        Parameters:
        ----------
        df_input: pd.DataFrame
        filter_on: str
            defines the used column
        Returns:
        ----------
        df_output: pd.DataFrame
            the result will be joined as a new column on the input data frame
    '''

    must_contain=set(['state','country',filter_on])
    assert must_contain.issubset(set(df_input.columns)), ' Erro in calc_filtered_data not all columns in data frame'

    df_output=df_input.copy() # we need a copy here otherwise the filter_on column will be overwritten

    pd_filtered_result=df_output[['state','country',filter_on]].groupby(['state','country']).apply(savgol_filter)#.reset_index()

    #print('--+++ after group by apply')
    #print(pd_filtered_result[pd_filtered_result['country']=='Germany'].tail())

    #df_output=pd.merge(df_output,pd_filtered_result[['index',str(filter_on+'_filtered')]],on=['index'],how='left')
    df_output=pd.merge(df_output,pd_filtered_result[[str(filter_on+'_filtered')]],left_index=True,right_index=True,how='left')
    #print(df_output[df_output['country']=='Germany'].tail())
    return df_output.copy()





def calc_doubling_rate(df_input,filter_on='confirmed'):
    ''' Calculate approximated doubling rate and return merged data frame

        Parameters:
        ----------
        df_input: pd.DataFrame
        filter_on: str
            defines the used column
        Returns:
        ----------
        df_output: pd.DataFrame
            the result will be joined as a new column on the input data frame
    '''

    must_contain=set(['state','country',filter_on])
    assert must_contain.issubset(set(df_input.columns)), ' Erro in calc_filtered_data not all columns in data frame'


    pd_DR_result= df_input.groupby(['state','country']).apply(rolling_reg,filter_on).reset_index()

    pd_DR_result=pd_DR_result.rename(columns={filter_on:filter_on+'_DR',
                             'level_2':'index'})

    #we do the merge on the index of our big table and on the index column after groupby
    df_output=pd.merge(df_input,pd_DR_result[['index',str(filter_on+'_DR')]],left_index=True,right_on=['index'],how='left')
    df_output=df_output.drop(columns=['index'])


    return df_output


if __name__ == '__main__':
    test_data_reg=np.array([2,4,6])
    result=get_doubling_time_via_regression(test_data_reg)
    print('the test slope is: '+str(result))

    pd_JH_data=pd.read_csv('data/processed/COVID_relational_confirmed.csv',sep=';',parse_dates=[0])
    pd_JH_data=pd_JH_data.sort_values('date',ascending=True).copy()

    #test_structure=pd_JH_data[((pd_JH_data['country']=='US')|
    #                  (pd_JH_data['country']=='Germany'))]

    pd_result_larg=calc_filtered_data(pd_JH_data)
    pd_result_larg=calc_doubling_rate(pd_result_larg)
    pd_result_larg=calc_doubling_rate(pd_result_larg,'confirmed_filtered')


    mask=pd_result_larg['confirmed']>100
    pd_result_larg['confirmed_filtered_DR']=pd_result_larg['confirmed_filtered_DR'].where(mask, other=np.NaN)
    pd_result_larg.to_csv('data/processed/COVID_final_set.csv',sep=';',index=False)
    print(pd_result_larg[pd_result_larg['country']=='Germany'].tail())


the test slope is: [2.]
            date state  country  confirmed  confirmed_filtered  confirmed_DR  \
34360 2020-09-10    no  Germany   258149.0            258018.2    160.722431   
34361 2020-09-11    no  Germany   259735.0            259374.2    156.332930   
34362 2020-09-12    no  Germany   260817.0            260732.0    194.577961   
34363 2020-09-13    no  Germany   261737.0            261946.8    260.502498   
34364 2020-09-14    no  Germany   263222.0            263161.6    217.817325   

       confirmed_filtered_DR  
34360             168.789051  
34361             184.661656  
34362             191.152480  
34363             202.662158  
34364             215.629569  


In [5]:
print(pd_result_larg[pd_result_larg['country']=='US'].tail())

            date state country  confirmed  confirmed_filtered  confirmed_DR  \
59956 2020-09-10    no      US  6396100.0           6402419.2    184.137066   
59957 2020-09-11    no      US  6443652.0           6440932.0    153.403356   
59958 2020-09-12    no      US  6485123.0           6479620.0    144.718219   
59959 2020-09-13    no      US  6519573.0           6518722.5    170.777062   
59960 2020-09-14    no      US  6553652.0           6557825.0    190.268334   

       confirmed_filtered_DR  
59956             180.980210  
59957             169.810423  
59958             166.863307  
59959             166.595103  
59960             166.708586  


SIR Model

In [6]:

from scipy.optimize import curve_fit
from scipy.integrate import odeint

def sir_simulations( confirmed_data, recovered_data, dates):
 
    duration_for_simulation = 30 # duration for simulations 

    confirmed_data = confirmed_data[(len(confirmed_data)-duration_for_simulation):]

    recovered_data = recovered_data[(len(recovered_data)- duration_for_simulation):]

    dates = dates[ len(dates)-duration_for_simulation: ]
    N = 1000000
    I_0 = confirmed_data[0]
    R_0 = recovered_data[0]
    S_0 = N - R_0 - I_0

    def SIR(y, t, beta, gamma):    
        S = y[0]
        I = y[1]
        R = y[2]
        return -beta*S*I/N, (beta*S*I)/N-(gamma*I), gamma*I

    def fit_odeint(t,beta, gamma):
        return odeint(SIR,(S_0,I_0,R_0), t, args = (beta,gamma))[:,1]

    t = np.arange(len(confirmed_data))
    params, cerr = curve_fit(fit_odeint,t, confirmed_data)
    beta,gamma = params
    prediction = list(fit_odeint(t,beta,gamma))


    fig = go.Figure()
    fig.add_trace(go.Scatter(x= dates, y= prediction,
                        mode='lines+markers',
                        name='Simulated'))
    fig.add_bar(x = dates, y= confirmed_data, name = "Actual")
    fig.update_layout(height = 800,
    title={
        'text':"SIR simulations",
        'x' : 0.5},
    xaxis_title="Date",
    yaxis_title="Infections")

    return fig

def sir_country_data_test(country):

    confirmed_cases =  list(confirmed_data.loc[f'{country}'])
    recovered_cases =  list(recovered_data.loc[f'{country}'])
    death_cases = list(deaths_data.loc[f'{country}'])

    sir_figure_country = sir_simulations( confirmed_cases, recovered_cases, dates)

#     confirmed = px.line(x = dates ,y = confirmed_cases , labels = { "x" : "Date", "y" : "Confirmed cases"}, height = 800)
#     confirmed.update_layout(title_text = " Confirmed cases" ,title_x=0.5 )


#     recovered = px.line(x = dates ,y = recovered_cases , labels = { "x" : "Date", "y" : "Recovered cases"}, height = 800)
#     recovered.update_layout(title_text = " Recovered cases" ,title_x=0.5)

#     deaths = px.line(x = dates ,y = death_cases , labels = { "x" : "Date", "y" : "Deaths"} ,height = 800)
#     deaths.update_layout(title_text = " Deaths" ,title_x=0.5 )

    return sir_figure_country

## 4 Visual Board

In [7]:
# %load src/visualization/visualize.py
import pandas as pd
import numpy as np

import dash
dash.__version__
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output,State

import plotly.graph_objects as go

import os
print(os.getcwd())
df_input_large=pd.read_csv('data/processed/COVID_final_set.csv',sep=';')


fig = go.Figure()



app = dash.Dash()
app.layout = html.Div([

    dcc.Markdown('''
    #  Applied Data Science on COVID-19 data

    The main aim of the project is to display the COVID-19 data of all the countries and SIR simulations.
    '''),

    dcc.Markdown('''
    ## Multi-Select Country for visualization
    '''),


    dcc.Dropdown(
        id='country_drop_down',
        options=[ {'label': each,'value':each} for each in df_input_large['country'].unique()],
        value=['US', 'Germany','Italy'], # which are pre-selected
        multi=True
    ),

    dcc.Markdown('''
        ## Select Timeline of confirmed COVID-19 cases or the approximated doubling time
        '''),


    dcc.Dropdown(
    id='doubling_time',
    options=[
        {'label': 'Timeline Confirmed ', 'value': 'confirmed'},
        {'label': 'Timeline Confirmed Filtered', 'value': 'confirmed_filtered'},
        {'label': 'Timeline Doubling Rate', 'value': 'confirmed_DR'},
        {'label': 'Timeline Doubling Rate Filtered', 'value': 'confirmed_filtered_DR'},
    ],
    value='confirmed',
    multi=False
    ),

    dcc.Graph(figure=fig, id='main_window_slope'), 
    
    dcc.Markdown('''
    #  SIR Simulation

    To simulate an SIR model based on existing data.

    '''),
    
    dcc.Dropdown(
        id='country_sir_drop_down',
        options=[ {'label': each,'value':each} for each in df_input_large['country'].unique()],
        value='Germany', # which are pre-selected
        multi=False
    ),
    
    dcc.Markdown('''
        ## Select country to view SIR simulation
        '''),
    
    dcc.Graph(figure=fig, id='sir_figure')
    
    

])



@app.callback(
    Output('main_window_slope', 'figure'),
    [Input('country_drop_down', 'value'),
    Input('doubling_time', 'value')])
def update_figure(country_list,show_doubling):


    if 'doubling_rate' in show_doubling:
        my_yaxis={'type':"log",
               'title':'Approximated doubling rate over 3 days (larger numbers are better #stayathome)'
              }
    else:
        my_yaxis={'type':"log",
                  'title':'Confirmed infected people (source johns hopkins csse, log-scale)'
              }


    traces = []
    for each in country_list:

        df_plot=df_input_large[df_input_large['country']==each]

        if show_doubling=='doubling_rate_filtered':
            df_plot=df_plot[['state','country','confirmed','confirmed_filtered','confirmed_DR','confirmed_filtered_DR','date']].groupby(['country','date']).agg(np.mean).reset_index()
        else:
            df_plot=df_plot[['state','country','confirmed','confirmed_filtered','confirmed_DR','confirmed_filtered_DR','date']].groupby(['country','date']).agg(np.sum).reset_index()
       #print(show_doubling)


        traces.append(dict(x=df_plot.date,
                                y=df_plot[show_doubling],
                                mode='markers+lines',
                                opacity=0.9,
                                name=each
                        )
                )

    return {
            'data': traces,
            'layout': dict (
                width=1280,
                height=720,

                xaxis={'title':'Timeline',
                        'tickangle':-45,
                        'nticks':20,
                        'tickfont':dict(size=14,color="#7f7f7f"),
                      },

                yaxis=my_yaxis
        )
         }
@app.callback(
    Output('sir_figure', 'figure'),
    [Input('country_sir_drop_down', 'value')])
def sir_country_data(country):

    confirmed_cases =  list(confirmed_data.loc[f'{country}'])
    recovered_cases =  list(recovered_data.loc[f'{country}'])
    death_cases = list(deaths_data.loc[f'{country}'])

    sir_figure_country = sir_simulations( confirmed_cases, recovered_cases, dates)

    return sir_figure_country
        

if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False)

C:\Users\vishw\ari\Lecture_Covid_19_data_analysis
Dash is running on http://127.0.0.1:8050/



UnsupportedOperation: not writable